In [8]:
'''MNIST to Numpy from http://g.sweyla.com/blog/2012/mnist-numpy/
   The only code I added was to convert labels into one hot encoding,
   change the dimensions of the numpy arrays containing the images to 2D from 3D,
   and to save the training and test data as pickle files'''

import pickle
import os, struct
from array import array as pyarray
import numpy as np
from numpy import append, array, int8, uint8, zeros

def load_mnist(dataset="training", digits=np.arange(10), path="."):
    """
    Loads MNIST files into 3D numpy arrays

    Adapted from: http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
    """

    if dataset == "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset == "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    N = len(ind)

    images = zeros((N, rows, cols), dtype=uint8)
    labels = zeros((N, 1), dtype=int8)
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((rows, cols))
        labels[i] = lbl[ind[i]]

    return images, labels

In [9]:
train_x, train_y = load_mnist(dataset="training", digits=np.arange(10), path=".")
test_x, test_y = load_mnist(dataset="testing", digits=np.arange(10), path=".")
print(train_x.shape)
print(test_x.shape)
num_pixels = train_x.shape[1]
train_x = np.reshape(train_x, (train_x.shape[0], num_pixels * num_pixels))
test_x = np.reshape(test_x, (test_x.shape[0], num_pixels * num_pixels))
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000, 784)
(60000, 1)
(10000, 784)
(10000, 1)


In [10]:
def convert_to_one_hot_encoding(y):    
    one_hot_y = np.zeros((y.shape[0],10))
    for i in range(y.shape[0]):
        index = y[i]
        one_hot_y[i][index] = 1
    return one_hot_y

new_train_y = convert_to_one_hot_encoding(train_y)
new_test_y = convert_to_one_hot_encoding(test_y)
print(train_y[0:10,:])
print(new_train_y[0:10,:])
print(test_y[0:10,:])
print(new_test_y[0:10,:])

[[5]
 [0]
 [4]
 [1]
 [9]
 [2]
 [1]
 [3]
 [1]
 [4]]
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]
[[7]
 [2]
 [1]
 [0]
 [4]
 [1]
 [4]
 [9]
 [5]
 [9]]
[[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


In [11]:
pickle.dump(train_x, open("MNIST_train_x.pkl", "wb"))
pickle.dump(new_train_y, open("MNIST_train_y.pkl", "wb"))
pickle.dump(test_x, open("MNIST_test_x.pkl", "wb"))
pickle.dump(new_test_y, open("MNIST_test_y.pkl", "wb"))